In [1]:
import os
os.chdir('/home/clemens/armed_conflict_avalanche/')

from workspace.utils import *
from voronoi_globe.interface import load_voronoi
from arcolanche.pipeline import *


In [2]:
conflict_type = "battles"
mesoscale = (32,453,3) #(dt, dx, gridix)
coarse = (32, 80, 3)
dt, dx, gridix = mesoscale


In [2]:
def calculate_significant_edges(self_edges, pair_edges, summary = False):
        
    significant_edges = [[] for _ in range(degree+1)]
    counts = np.zeros(degree+1)
    
    #self edges
    for poly, (te, te_shuffle) in self_edges.items():
        if (te > te_shuffle).mean() >= (95 / 100):
            significant_edges[0].append([(poly, poly), te])
    counts[0] = len(self_edges)

    #pair edges
    for pair, (te, te_shuffle) in pair_edges.items():
        d = pair[2]
        if 1 <= d <= degree:
            counts[d] += 1
            if (te > te_shuffle).mean() >= (95/100): #threshold!
                significant_edges[d].append([(pair[0], pair[1]), te])            
    
    #ratio dataframe 
    if summary:
        df = pd.DataFrame(
            {
                "Degree": np.arange(degree+1),
                "Total edges": counts,
                "Significant edges": [len(edges) for edges in significant_edges],
            }
        )
        df["Ratio"] = round(df["Significant edges"] / df["Total edges"], 3)
        
        return df, significant_edges
    
    else:
        return significant_edges

In [6]:
degree = 1
ava_1 = Avalanche(dt = dt, dx = dx, gridix=gridix, degree = degree, triples=False, setup=True)
ava_1_self = ava_1.self_edges
ava_1_pair = ava_1.pair_edges

df_1, sign_edges = calculate_significant_edges(ava_1_self, ava_1_pair, summary = True)

#triples
ava_2 = Avalanche(dt = dt, dx = dx, gridix=gridix, degree = degree, triples=True, setup=True)
ava_2_self = ava_2.self_edges
ava_2_pair = ava_2.pair_edges

#pair edges
significant_triples = []
for triplet, (te, te_shuffle) in ava_2_pair.items():
    if (te > te_shuffle).mean() >= (95/100): #threshold!
        significant_triples.append([(triplet[0], triplet[1], triplet[2]), te])            

print(df_1)
print(len(significant_triples)/len(ava_2_pair))


/home/clemens/miniconda3/envs/conflict/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/home/clemens/armed_conflict_avalanche/arcolanche/triple_transfer_entropy_func.py:80: RuntimeWarning: invalid value encountered in true_divide
  p_xt1_given_zt_yt_xt = joint/p_zt_yt_xt[:, :, :, np.newaxis] #p(xt1|zt, yt, xt) = p(xt1, zt, yt, xt)/p(zt, yt, xt), add axis for explicit broadasting: colwise!


   Degree  Total edges  Significant edges  Ratio
0       0       3085.0                759  0.246
1       1      11390.0               2062  0.181
0.30132095680114246


In [3]:
import os
from workspace.utils import save_pickle

def save_avalanche(ava, conflict_type, gridix, dt, dx, triplets):
    ava_box = [[tuple(i) for i in ava.time_series.loc[a].drop_duplicates().values[:, ::-1]] for a in ava.avalanches]
    ava_event = ava.avalanches

    if triplets:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/triplets"
        if not os.path.exists(path):
            os.makedirs(path)
            
        save_pickle(["ava_box", "ava_event"], f"{path}/triplet_ava_{str(dt)}_{str(dx)}.p", True)
    
    else:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/pairs"
        if not os.path.exists(path):
            os.makedirs(path)
        save_pickle(["ava_box", "ava_event"], f"{path}/pair_ava_{str(dt)}_{str(dx)}.p", True)

    


In [2]:
save_avalanche(ava_1, conflict_type, gridix, dt, dx, triplets=False)



NameError: name 'save_avalanche' is not defined

In [4]:
#todo read pickle
def open_avalanche(conflict_type, gridix, dt, dx, triplets):
    
    if triplets:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/triplets"
        ava_box, ava_event = pickle.load(f"{path}/triplet_ava_{str(dt)}_{str(dx)}.p")
    
    else:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/pairs"
        ava_box, ava_event = pickle.load(f"{path}/pair_ava_{str(dt)}_{str(dx)}.p")

    return ava_box, ava_event

In [8]:
def load_avalanche(conflict_type, gridix, dt, dx, triplets):
    if triplets:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/triplets/triplet_ava_{str(dt)}_{str(dx)}.p"
    else:
        path = f"Results/avalanches/{conflict_type}/gridix_{gridix}/pairs/pair_ava_{str(dt)}_{str(dx)}.p"
    
    with open(path, 'rb') as file:
        data = pickle.load(file)
    return data

# Example usage:

data = load_avalanche(path)
